# 🏠 House Price Prediction - Exploratory Data Analysis (EDA)

## VietAI - Foundations of Machine Learning Final Project

**Mục tiêu:**
- Hiểu rõ cấu trúc và đặc điểm của bộ dữ liệu House Prices
- Phân tích thống kê mô tả các biến
- Phát hiện missing values và outliers
- Trực quan hóa mối tương quan giữa các biến

**Dataset:** [Kaggle House Prices Competition](https://www.kaggle.com/c/house-prices-advanced-regression-techniques)


In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
import os

# Settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Create directories
os.makedirs('../data/raw', exist_ok=True)
os.makedirs('../data/processed', exist_ok=True)
os.makedirs('../reports', exist_ok=True)
os.makedirs('../models', exist_ok=True)

# Custom color palette
COLORS = {
    'primary': '#2E86AB',
    'secondary': '#A23B72',
    'accent': '#F18F01',
    'success': '#C73E1D',
    'dark': '#3B1F2B'
}

print("✅ Libraries imported successfully!")


✅ Libraries imported successfully!


## 1. Load Data

**Lưu ý:** Tải dữ liệu từ [Kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data) và đặt trong thư mục `../data/raw/`


In [4]:
# Load training data
train_path = '../data/raw/train.csv'
test_path = '../data/raw/test.csv'

try:
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)
    print(f"✅ Training data loaded: {train_df.shape[0]} rows, {train_df.shape[1]} columns")
    print(f"✅ Test data loaded: {test_df.shape[0]} rows, {test_df.shape[1]} columns")
except FileNotFoundError:
    print("⚠️ Data files not found!")
    print("Please download data from: https://www.kaggle.com/c/house-prices-advanced-regression-techniques")
    print("And place train.csv and test.csv in ../data/raw/ folder")


⚠️ Data files not found!
Please download data from: https://www.kaggle.com/c/house-prices-advanced-regression-techniques
And place train.csv and test.csv in ../data/raw/ folder


In [5]:
# First look at the data
train_df.head(10)


NameError: name 'train_df' is not defined

In [ ]:
# Data info
print("="*60)
print("DATA INFO")
print("="*60)
train_df.info()


## 2. Thống Kê Mô Tả (Descriptive Statistics)


In [ ]:
# Numerical columns statistics
print("📊 THỐNG KÊ CÁC BIẾN SỐ")
print("="*80)
train_df.describe().T


In [ ]:
# Categorical columns
categorical_cols = train_df.select_dtypes(include=['object']).columns
numerical_cols = train_df.select_dtypes(include=[np.number]).columns

print(f"\n📌 Số lượng biến số (Numerical): {len(numerical_cols)}")
print(f"📌 Số lượng biến phân loại (Categorical): {len(categorical_cols)}")
print(f"\n🔢 Biến số: {list(numerical_cols[:10])}...")
print(f"\n📝 Biến phân loại: {list(categorical_cols[:10])}...")


In [ ]:
# Target variable (SalePrice) statistics
print("\n🎯 BIẾN MỤC TIÊU: SalePrice")
print("="*50)
print(train_df['SalePrice'].describe())
print(f"\n📈 Skewness: {train_df['SalePrice'].skew():.4f}")
print(f"📊 Kurtosis: {train_df['SalePrice'].kurtosis():.4f}")


## 3. Phân Tích Missing Values


In [ ]:
# Missing values analysis
def analyze_missing_values(df, name="Dataset"):
    """Analyze missing values in a DataFrame."""
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    
    missing_df = pd.DataFrame({
        'Column': missing.index,
        'Missing Count': missing.values,
        'Missing %': missing_pct.values
    })
    
    missing_df = missing_df[missing_df['Missing Count'] > 0]
    missing_df = missing_df.sort_values('Missing %', ascending=False)
    
    print(f"\n🔍 {name} - MISSING VALUES ANALYSIS")
    print("="*60)
    print(f"Tổng số cột có missing values: {len(missing_df)}")
    print(f"Tổng số giá trị missing: {missing_df['Missing Count'].sum()}")
    
    return missing_df

missing_train = analyze_missing_values(train_df, "Training Data")
missing_train.head(20)


In [ ]:
# Visualize missing values
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart of missing values
if len(missing_train) > 0:
    top_missing = missing_train.head(15)
    ax1 = axes[0]
    bars = ax1.barh(top_missing['Column'], top_missing['Missing %'], 
                    color=COLORS['primary'], edgecolor=COLORS['dark'])
    ax1.set_xlabel('Phần trăm Missing (%)', fontsize=12)
    ax1.set_title('Top 15 Cột Có Missing Values', fontsize=14, fontweight='bold')
    ax1.invert_yaxis()
    
    # Add value labels
    for bar, pct in zip(bars, top_missing['Missing %']):
        ax1.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2, 
                f'{pct:.1f}%', va='center', fontsize=10)

# Heatmap of missing values
ax2 = axes[1]
cols_with_missing = missing_train['Column'].tolist()[:20]
if cols_with_missing:
    msno_data = train_df[cols_with_missing].isnull()
    sns.heatmap(msno_data.T, cbar=True, ax=ax2, cmap='YlOrRd')
    ax2.set_title('Missing Values Pattern', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Sample Index')

plt.tight_layout()
plt.savefig('../reports/missing_values.png', dpi=150, bbox_inches='tight')
plt.show()


## 4. Phân Tích Biến Mục Tiêu (Target Variable)


In [ ]:
# Target variable distribution
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Histogram
ax1 = axes[0]
ax1.hist(train_df['SalePrice'], bins=50, color=COLORS['primary'], 
         edgecolor=COLORS['dark'], alpha=0.7)
ax1.axvline(train_df['SalePrice'].mean(), color=COLORS['secondary'], 
            linestyle='--', linewidth=2, label=f'Mean: ${train_df["SalePrice"].mean():,.0f}')
ax1.axvline(train_df['SalePrice'].median(), color=COLORS['accent'], 
            linestyle='--', linewidth=2, label=f'Median: ${train_df["SalePrice"].median():,.0f}')
ax1.set_xlabel('SalePrice ($)', fontsize=12)
ax1.set_ylabel('Frequency', fontsize=12)
ax1.set_title('Phân Phối Giá Nhà', fontsize=14, fontweight='bold')
ax1.legend()

# Log-transformed
ax2 = axes[1]
log_price = np.log1p(train_df['SalePrice'])
ax2.hist(log_price, bins=50, color=COLORS['secondary'], 
         edgecolor=COLORS['dark'], alpha=0.7)
ax2.set_xlabel('Log(SalePrice)', fontsize=12)
ax2.set_ylabel('Frequency', fontsize=12)
ax2.set_title('Phân Phối Log-Transformed', fontsize=14, fontweight='bold')

# Q-Q Plot
ax3 = axes[2]
stats.probplot(log_price, dist="norm", plot=ax3)
ax3.set_title('Q-Q Plot (Log SalePrice)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('../reports/target_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📊 Skewness gốc: {train_df['SalePrice'].skew():.4f}")
print(f"📊 Skewness sau log transform: {log_price.skew():.4f}")


## 5. Phân Tích Tương Quan (Correlation Analysis)


In [ ]:
# Correlation with SalePrice
correlations = train_df.select_dtypes(include=[np.number]).corr()['SalePrice'].sort_values(ascending=False)

print("🔗 TOP 15 BIẾN TƯƠNG QUAN CAO NHẤT VỚI SALEPRICE")
print("="*50)
for col, corr in correlations.head(16).items():
    if col != 'SalePrice':
        print(f"{col:25s}: {corr:.4f}")


In [ ]:
# Correlation heatmap for top features
top_corr_features = correlations.head(12).index.tolist()

fig, ax = plt.subplots(figsize=(12, 10))
corr_matrix = train_df[top_corr_features].corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', 
            cmap='RdYlBu_r', center=0, linewidths=0.5,
            square=True, ax=ax)

ax.set_title('Ma Trận Tương Quan - Top Features', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('../reports/correlation_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
# Scatter plots for top correlated features
top_features = ['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 
                'FullBath', 'YearBuilt']

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for i, feature in enumerate(top_features):
    ax = axes[i]
    ax.scatter(train_df[feature], train_df['SalePrice'], 
               alpha=0.5, c=COLORS['primary'], edgecolors=COLORS['dark'], linewidth=0.5)
    
    # Add trend line
    valid_mask = train_df[feature].notna()
    z = np.polyfit(train_df.loc[valid_mask, feature], 
                   train_df.loc[valid_mask, 'SalePrice'], 1)
    p = np.poly1d(z)
    x_line = np.linspace(train_df[feature].min(), train_df[feature].max(), 100)
    ax.plot(x_line, p(x_line), color=COLORS['secondary'], linewidth=2, label='Trend')
    
    corr = train_df[feature].corr(train_df['SalePrice'])
    ax.set_xlabel(feature, fontsize=11)
    ax.set_ylabel('SalePrice ($)', fontsize=11)
    ax.set_title(f'{feature} vs SalePrice\n(r = {corr:.3f})', fontsize=12, fontweight='bold')

plt.suptitle('Mối Quan Hệ Giữa Top Features và Giá Nhà', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('../reports/scatter_plots.png', dpi=150, bbox_inches='tight')
plt.show()


## 6. Phân Tích Biến Phân Loại (Categorical Features)


In [ ]:
# Analyze categorical features vs SalePrice
important_cat_features = ['Neighborhood', 'ExterQual', 'KitchenQual', 
                          'BldgType', 'HouseStyle', 'SaleCondition']

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for i, feature in enumerate(important_cat_features):
    ax = axes[i]
    
    # Order by median SalePrice
    order = train_df.groupby(feature)['SalePrice'].median().sort_values(ascending=False).index
    
    sns.boxplot(data=train_df, x=feature, y='SalePrice', order=order,
                palette='viridis', ax=ax)
    
    ax.set_xlabel(feature, fontsize=11)
    ax.set_ylabel('SalePrice ($)', fontsize=11)
    ax.set_title(f'SalePrice by {feature}', fontsize=12, fontweight='bold')
    ax.tick_params(axis='x', rotation=45)

plt.suptitle('Phân Tích Giá Nhà Theo Biến Phân Loại', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('../reports/categorical_analysis.png', dpi=150, bbox_inches='tight')
plt.show()


## 7. Phát Hiện Outliers


In [ ]:
# Detect outliers using IQR method
def detect_outliers_iqr(df, column):
    """Detect outliers using IQR method."""
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower) | (df[column] > upper)]
    return outliers, lower, upper

# Analyze outliers for key numerical features
key_features = ['SalePrice', 'GrLivArea', 'LotArea', 'TotalBsmtSF', 'GarageArea']

print("🔍 OUTLIERS ANALYSIS (IQR Method)")
print("="*60)

for feature in key_features:
    if feature in train_df.columns:
        outliers, lower, upper = detect_outliers_iqr(train_df, feature)
        print(f"\n{feature}:")
        print(f"  - Bounds: [{lower:.2f}, {upper:.2f}]")
        print(f"  - Number of outliers: {len(outliers)} ({len(outliers)/len(train_df)*100:.2f}%)")


In [ ]:
# Identify specific outliers in GrLivArea
fig, ax = plt.subplots(figsize=(10, 6))

ax.scatter(train_df['GrLivArea'], train_df['SalePrice'], 
           c=COLORS['primary'], alpha=0.6, edgecolors=COLORS['dark'])

# Highlight potential outliers
outlier_mask = (train_df['GrLivArea'] > 4000) & (train_df['SalePrice'] < 300000)
ax.scatter(train_df.loc[outlier_mask, 'GrLivArea'], 
           train_df.loc[outlier_mask, 'SalePrice'],
           c=COLORS['success'], s=100, marker='x', linewidths=3,
           label=f'Potential Outliers ({outlier_mask.sum()})')

ax.set_xlabel('GrLivArea (sq ft)', fontsize=12)
ax.set_ylabel('SalePrice ($)', fontsize=12)
ax.set_title('GrLivArea vs SalePrice - Outlier Detection', fontsize=14, fontweight='bold')
ax.legend()

plt.tight_layout()
plt.savefig('../reports/grlivarea_outliers.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n⚠️ Identified {outlier_mask.sum()} potential outliers with large area but low price")


## 8. Kết Luận EDA

### Những phát hiện chính:
1. **Chất lượng tổng thể (OverallQual)** là yếu tố quan trọng nhất ảnh hưởng đến giá nhà
2. **Diện tích** (GrLivArea, TotalBsmtSF) có mối tương quan mạnh với giá
3. **Missing values** cần được xử lý cẩn thận, đặc biệt các cột liên quan đến tiện ích đặc biệt (Pool, Fence, Alley)
4. **Log transformation** nên được áp dụng cho biến SalePrice để giảm skewness
5. **Feature Engineering** có thể tạo ra các biến mới có giá trị như TotalSF, HouseAge

### Bước tiếp theo:
- Data Preprocessing & Feature Engineering
- Model Training & Evaluation
